In [ ]:
import sys, os
import glob
import weakref

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
import numpy as np
import pandas as pd
from scipy.signal import convolve2d

from bio2byte.mdutils.utils.BBanalysis import BBDihedrals, stack_uneven, boltzmann_energy

# Font size and familiy
matplotlib.rcParams['font.size']          =   8
matplotlib.rcParams['axes.labelsize']     =   8
matplotlib.rcParams["mathtext.fontset"]   =    "dejavusans"
matplotlib.rcParams['axes.linewidth']     =   1
matplotlib.rcParams['lines.linewidth']    =   .9

def logprob(input_array):
    arr = input_array.copy()
    arr[arr == 0] = np.nan
    #arr[np.isnan(arr)] = 1 - p_value ** (1. / sample_size)
    return -np.log(arr)

%matplotlib inline

In [ ]:
peptides = {
    os.path.basename(mddir): BBDihedrals(mddir+'/analy/dihedrals.csv')
    for mddir in sorted(glob.glob("../5peptides/gg*gg")) if os.path.isfile(mddir+'/analy/dihedrals.csv')
}
peptide_names = [n for n in peptides.keys()]

for bbobj in peptides.values():
    bbobj.histogram_bins = np.linspace(-180, 180, 41)

# Normalization over all peptides
data  = stack_uneven([pept.scatter[resid] for pept in peptides.values() for resid in pept.iterresids()])
norma = matplotlib.colors.Normalize(vmin=np.nanmin(data[:,2,:]), 
                                    vmax=np.nanmax(data[:,2,:]))
lognorma = matplotlib.colors.Normalize(vmin=-np.log(np.nanmax(data[:,2,:])), 
                                       vmax=-np.log(np.nanmin(data[:,2,:])))
del data

In [ ]:
new_params = {
    "axes.labelsize": 8,
    "axes.linewidth": .8,
    'boxplot.flierprops.linewidth': 2.0,
    'font.size': 8,
    "grid.linewidth": .8,
    'lines.linewidth': .8,
    "patch.linewidth": .8,
    'xtick.labelsize': 8,
    "xtick.major.width": .8,
    "xtick.minor.width": .5,
    'ytick.labelsize': 8,
    "ytick.major.width": .8,
    "ytick.minor.width": .5,
}
bak_params = {k: matplotlib.rcParams[k] for k in new_params.keys()}
matplotlib.rcParams.update(new_params)


for peptide, bbobj in peptides.items():
    
    x,y,c = bbobj.scatter[4]
    c = logprob(c)
    
    fig, ax = plt.subplots(1,1, figsize=(1,1), dpi=300,  gridspec_kw={"left":0., "right":1., "bottom":0., "top":1.})
    fig.patch.set_alpha(0)
    
    #ax.set_title(peptide)
    ax.set_aspect("equal")
    ax.grid(True, "major", "both", color='grey', linestyle="--", alpha=.8, linewidth=.7, zorder=1)
    im = ax.scatter(x[:10], y[:10], s=1, c=c[:10], norm=lognorma, marker='.', cmap=cm.turbo)
    
    ax.text(135, -170, peptide[2:5], ha="center", va="bottom")
    
    #ax.set_ylabel("$\psi$"); ax.set_xlabel("$\phi$")
    ax.set_xlim([-180,180])
    ax.set_xticks(np.arange(-180,181,90)); ax.set_xticks(np.arange(-180,181,45), minor=True)
    ax.set_xticklabels([])
    ax.set_ylim([-180,180])
    ax.set_yticks(np.arange(-180,181,90)); ax.set_yticks(np.arange(-180,181,45), minor=True)
    ax.set_yticklabels([])
    
    # cbar = plt.colorbar(im, ax=ax, fraction=.6)
    # cbar.ax.set_yticks(np.arange(5,15,5))
    # cbar.ax.set_yticks(np.arange(3,14), minor=True)
    # cbar.ax.get_yaxis().labelpad = 15
    # cbar.ax.set_ylabel('$\Delta G \ \ [k_{B} T]$')  
    # fig.savefig(f"figures/rama_colorbar.pdf", dpi=300, bbox_inches="tight")
    
    fig.savefig(f"figures/rama_{peptide}.pdf", dpi=300, bbox_inches="tight")
    plt.clf()
    break
    
matplotlib.rcParams.update(bak_params)

In [ ]:
#
# ::: Create kernel for smoothing
#
x, y = np.meshgrid(np.linspace(-2,2,18), np.linspace(-2,2,18))
mu, sigma = 0., .9
knl = np.exp(- (np.sqrt(x**2 + y**2) - mu)**2 / (2 * sigma**2))
knl /= np.sum(knl)
# _fig, _ax = plt.subplots(1,1, figsize=(1.2,1))
# im = _ax.imshow(knl)
# plt.colorbar(im)
# plt.show()


bbobj = peptides["ggTRPgg"]
hist = bbobj.hist[4]
hist = logprob(hist)
hist[np.isnan(hist)]= 15
hist = np.repeat(hist, repeats=5, axis=0)
hist = np.repeat(hist, repeats=5, axis=1)

i, j = hist.shape
ihist = np.linspace(-180, 180, 2*i+1)[1::2]
jhist = np.linspace(-180, 180, 2*j+1)[1::2]
di, dj = ihist[1] - ihist[0], jhist[1] - jhist[0]
ihist, jhist = np.meshgrid(ihist, jhist, indexing="ij")

smoothed = convolve2d(hist, knl, mode="full", boundary="wrap")
ismooth, jsmooth = smoothed.shape
ismooth = np.linspace(-.5*ismooth*di, .5*ismooth*di, 2*ismooth+1)[1::2]
jsmooth = np.linspace(-.5*jsmooth*dj, .5*jsmooth*dj, 2*jsmooth+1)[1::2]
ismooth, jsmooth = np.meshgrid(ismooth, jsmooth, indexing="ij")

new_params = {
    "axes.labelsize": 8,
    "axes.linewidth": .8,
    'boxplot.flierprops.linewidth': 2.0,
    'font.size': 8,
    "grid.linewidth": .8,
    'lines.linewidth': .8,
    "patch.linewidth": .8,
    'xtick.labelsize': 8,
    "xtick.major.width": .8,
    "xtick.minor.width": .5,
    'ytick.labelsize': 8,
    "ytick.major.width": .8,
    "ytick.minor.width": .5,
}
bak_params = {k: matplotlib.rcParams[k] for k in new_params.keys()}
matplotlib.rcParams.update(new_params)

#
# ::: Plot the whole thing
#
fig, ax = plt.subplots(1,1, figsize=(1,1), dpi=300,  gridspec_kw={"left":0., "right":1., "bottom":0., "top":1.})
fig.patch.set_alpha(0)

#ax.set_title(peptide)
ax.set_aspect("equal")
ax.grid(True, "major", "both", color='grey', linestyle="--", alpha=.8, linewidth=.7, zorder=1)
#im = ax.imshow(hist.T, cmap=cm.turbo, norm=lognorma, extent=(-180,180,180,-180))
#cntr = ax.contour(ihist, jhist, hist, levels=[5.2, 7, 11], colors=["0.1", "0.5", "0.7"], linewidths=[.6, .5, .5])
cntr = ax.contour(ismooth, jsmooth, smoothed, levels=[5.2, 7, 11], cmap=cm.binary_r, norm=matplotlib.colors.LogNorm(5,14),
                  linestyles=["-", "-", "-"], linewidths=[1, 1, 1], zorder=2)

ax.set_xlim([-180,180])
ax.set_xticks(np.arange(-180,181,90)); ax.set_xticks(np.arange(-180,181,45), minor=True)
ax.set_xticklabels([])
ax.set_ylim([-180,180])
ax.set_yticks(np.arange(-180,181,90)); ax.set_yticks(np.arange(-180,181,45), minor=True)
ax.set_yticklabels([])

fig.savefig(f"figures/rama_abstract.png", dpi=300, bbox_inches="tight")


matplotlib.rcParams.update(bak_params)

# All-in-one figure

In [ ]:
residues = [
    'ALA', 'ARG', 'ASN', 'ASP', 'ASH', 
    'CYS', 'GLN', 'GLU', 'GLH', 'GLY',
    'HIS', 'HIP', 'ILE', 'LEU', 'LYS',
    'PRO', 'TRP', 'VAL', 'MET', 'PHE', 
    'SER', 'S1P', 'SEP',
    'THR', 'T1P', 'TPO',
    'TYR', 'Y1P', 'PTR'
]

residues20 = [
    'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 
    'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 
    'LEU', 'LYS', 'PRO', 'SER', 'THR', 
    'TRP', 'TYR', 'VAL', 'MET', 'PHE'
]

In [ ]:
i,j = 5,4
fig, axs = plt.subplots(i,j, figsize=(.666*j, .666*i), dpi=300, sharex=True, sharey=True,
                        gridspec_kw={"wspace": 0., "hspace": 0., "left":0., "right":1., "bottom":0., "top":1.},)
fig.patch.set_alpha(0)

for i, (ax, res) in enumerate(zip(axs.flatten(), residues20)):
    
    x,y,c = peptides[f"gg{res}gg"].scatter[4]
    c = logprob(c)
    
    ax.set_aspect("equal")
    ax.grid(True, "both", "both", color="grey", linewidth=.4, linestyle="--", alpha=.8)
    ax.scatter(x[:], y[:], s=1, c=c[:], norm=lognorma, marker='.', cmap="turbo")
    ax.text(135, -155, res, color="k", ha="center", va="center", size=5, zorder=9) #if res != "GLY" else "w"
    
    # if i%4 == 0:
    #     ax.set_ylabel("$\psi \ [°]$")
    # if i >= 15:
    #     ax.set_xlabel("$\phi \ [°]$")

ax.set_xlim([-180., 180.])
ax.set_xticks(np.arange(-180,181,180)); ax.set_xticks(np.arange(-180,181,90), minor=True)
ax.set_xticklabels([])
ax.set_ylim([-180., 180.])
ax.set_yticks(np.arange(-180,181,180)); ax.set_yticks(np.arange(-180,181,90), minor=True)
ax.set_yticklabels([])

fig.savefig(f"figures/rama5_canonicalA.png", dpi=300, bbox_inches="tight")

In [ ]:
fig, axs = plt.subplots(3,4, figsize=(4.2, 3.6), dpi=96, sharex=True, sharey=True,
                        gridspec_kw={"wspace": 0., "hspace": 0., "left":0., "right":1., "bottom":0., "top":1., "width_ratios": [2,2,2,1]},)
#fig.set_facecolor("w")
#fig.set_alpha(1)

for i, ax, res in zip(range(9), axs[:,:3].flatten(), residues[20:]):
    
    x,y,c = peptides[f"gg{res}gg"].scatter[4]
    c = logprob(c)
    
    ax.grid(True, "both", "both", color="k", linewidth=.7, linestyle="--", alpha=.5)
    im = ax.scatter(x, y, s=1, c=c, norm=lognorma, marker='.', cmap="turbo")
    
    #ax.text(135, 135, res, color="yellow", weight="bold", ha="center", va="center", zorder=9)
    ax.text(135, -155, res, color="k", ha="center", va="center", zorder=9)
    
    if i%3 == 0:
        ax.set_ylabel("$\psi \ [°]$")
    if i > 5:
        ax.set_xlabel("$\phi \ [°]$")

ax.set_xlim([-180., 180.])
ax.set_xticks(np.arange(-180,181,180)); ax.set_xticks(np.arange(-180,181,90), minor=True)
ax.set_xticklabels([])
ax.set_ylim([-180., 180.])
ax.set_yticks(np.arange(-180,181,180)); ax.set_yticks(np.arange(-180,181,90), minor=True)
ax.set_yticklabels([])

for ax in axs[:,3]:
    ax.set_axis_off()
    
cbar = plt.colorbar(im, ax=axs[:,3])
cbar.set_label(r"$\Delta G \ [k_{B} T]$")
fig.savefig(f"figures/rama5_allB.png", dpi=150, bbox_inches="tight")